# <정류장 별 일일 평균 이용객 수>

## 사용 데이터

### - 세종시 버스 정류장 데이터
### - 버스 정류장 별 이용객 수 데이터

## 전처리 목표와 과정

### 목표 : 정류장 위치와 일일 평균 이용객수를 하나의 데이터로 생성
### 1. 상행, 하행 버스 정류장은 위도와 경도 평균값을 계산
### 2. 정류장명을 기준으로 조인
### 3. 세종시 내 도심과 부도심을 기준으로 정류장을 필터링
### 4. Min-Max 스케일링을 적용하여 '합계_정규화' 컬럼 추가

In [ ]:
import pandas as pd
from matplotlib.path import Path
from sklearn.preprocessing import MinMaxScaler

# 데이터 로드
sejong_bus_stops = pd.read_csv('/Users/Desktop/gong/sejong_bus_stops.csv', encoding='euc-kr')
average_data = pd.read_csv('/Users/Desktop/gong/average_data.csv', encoding='euc-kr')

# 1. '정류장명'을 기준으로 위도와 경도의 평균값을 계산
sejong_bus_stops_grouped = sejong_bus_stops.groupby('정류장명').agg({
    '위도': 'mean',
    '경도': 'mean'
}).reset_index()

# 2. '정류장명'을 기준으로 조인 (위도, 경도 추가)
merged_data = pd.merge(average_data, sejong_bus_stops_grouped, on='정류장명', how='left')

# 3. 성공한 조인과 실패한 조인을 구분 (위도가 NaN인 경우 실패)
successful_joins = merged_data[merged_data['위도'].notna()]
failed_joins = merged_data[merged_data['위도'].isna()]

# 4. 성공 및 실패한 항목 개수 확인
successful_count = len(successful_joins)
failed_count = len(failed_joins)

# 5. 성공한 항목과 실패한 항목을 각각 파일로 저장
successful_joins.to_csv('/Users/Desktop/gong/successful_joins_with_avg_lat_lon.csv', index=False, encoding='euc-kr')
failed_joins.to_csv('/Users/Desktop/gong/failed_joins.csv', index=False, encoding='euc-kr')

# 결과 출력
print(f"조인에 성공한 항목 수: {successful_count}")
print(f"조인에 실패한 항목 수: {failed_count}")

# 좌표 경계 설정
coordinate_bounds_1 = {
    'latitudes': [36.5231, 36.4973, 36.4738, 36.46462, 36.47581, 36.48788, 36.50058, 36.50624, 36.53241],
    'longitudes': [127.22716, 127.23064, 127.24508, 127.26714, 127.30464, 127.33417, 127.33752, 127.31366, 127.27536]
}

# 6. 좌표 경계 필터링
path = Path(list(zip(coordinate_bounds_1['longitudes'], coordinate_bounds_1['latitudes'])))
points = sejong_bus_stops[['경도', '위도']].values  # 경도, 위도의 순서가 중요함
mask = path.contains_points(points)
filtered_bus_stops = sejong_bus_stops[mask]

# 7. Min-Max 스케일링을 적용하여 '합계_정규화' 컬럼 추가
if '합계' in filtered_bus_stops.columns:
    scaler = MinMaxScaler()
    filtered_bus_stops['합계_정규화'] = scaler.fit_transform(filtered_bus_stops[['합계']])
    print("'합계' 열의 Min-Max 스케일링이 완료되었습니다.")
else:
    print("데이터에 '합계' 열이 존재하지 않습니다. 확인해주세요.")

# 8. 필터링된 데이터 저장
filtered_bus_stops.to_csv('/Users/Desktop/gong/bus_total_filtered_corrected.csv', 
                          index=False, encoding='utf-8-sig')

# 결과 출력
print(f"필터링된 데이터를 '/Users/Desktop/gong/bus_total_filtered_corrected.csv'에 저장했습니다.")


# <학교 별 재학생 수>

## 사용 데이터

### - 초, 중, 고, 특수 학교 별 위도 및 경도 데이터
### - 초, 중, 고, 특수 학교 별 학생 수 데이터

## 전처리 목표와 과정

### 목표 : 학교명, 위도, 경도, 학생 수를 하나의 데이터로 생성
### 1. 필요한 컬럼만 남기고 제거
### 2. 위치 데이터와 학생 수 데이터 조인
### 3. 조인 된 초, 중, 고, 특수 학교 데이터를 하나의 파일로 생성

In [ ]:
# 초등학교 위치 전처리
esl = esl.drop(columns=['시도교육청', '학교급코드', '설립구분', '학교특성', '분교여부', '설립유형', '전화번호', '팩스번호', '홈페이지 주소',
                        '지역','주야구분', '개교기념일', '설립일', '주소내역', '우편번호', '남녀공학 구분', '상세주소내역', '학교도로명 우편번호', '폐교일자'])
esl


# 중학교 위치 전처리
msl = msl.drop(columns=['시도교육청', '학교급코드', '설립구분', '학교특성', '분교여부', '설립유형', '전화번호', '팩스번호', '홈페이지 주소',
                        '지역','주야구분', '개교기념일', '설립일', '주소내역', '우편번호', '남녀공학 구분', '상세주소내역', '학교도로명 우편번호', '폐교일자'])
msl


# 고등학교 위치 전처리
hsl = hsl.drop(columns=['시도교육청', '학교급코드', '설립구분', '학교특성', '분교여부', '설립유형', '전화번호', '팩스번호', '홈페이지 주소',
                        '지역','주야구분', '개교기념일', '설립일', '주소내역', '우편번호', '남녀공학 구분', '상세주소내역', '학교도로명 우편번호', '폐교일자'])
hsl


# 특수학교 위치 전처리
ssl = ssl.drop(columns=['시도교육청', '학교급코드', '설립구분', '학교특성', '분교여부', '설립유형', '전화번호', '팩스번호', '홈페이지 주소',
                        '지역','주야구분', '개교기념일', '설립일', '주소내역', '우편번호', '남녀공학 구분', '상세주소내역', '학교도로명 우편번호', '폐교일자'])
ssl

In [ ]:
# 초등 학생수
esc = esc.drop(columns=['시도교육청', '지역', '학교급코드', '설립구분', '제외여부' ,'제외사유',
                        '1학년(남)', '1학년(여)', '2학년(남)', '2학년(여)', '3학년(남)', '3학년(여)',
                        '4학년(남)', '4학년(여)', '5학년(남)', '5학년(여)', '6학년(남)', '6학년(여)',
                        '특수학급(남)', '특수학급(여)', '순회학급(남)', '순회학급(여)', '계(남)', '계(여)'])
esc


# 중등 학생수
msc = msc.drop(columns=['시도교육청', '지역', '학교급코드', '설립구분', '제외여부' ,'제외사유',
                        '1학년(남)', '1학년(여)', '2학년(남)', '2학년(여)', '3학년(남)', '3학년(여)',
                        '특수학급(남)', '특수학급(여)', '순회학급(남)', '순회학급(여)', '계(남)', '계(여)'])
msc


# 고등 학생수
hsc = hsc.drop(columns=['시도교육청', '지역', '학교급코드', '설립구분', '제외여부' ,'제외사유',
                        '1학년(남)', '1학년(여)', '2학년(남)', '2학년(여)', '3학년(남)', '3학년(여)',
                        '특수학급(남)', '특수학급(여)', '순회학급(남)', '순회학급(여)', '계(남)', '계(여)'])
hsc


# 특수 학생수
ssc = ssc.drop(columns=['x1', 'x2', 'x3', 'x4', 'x5', 'x6', 'x7', 'x8', 'x9',
                        'z1', 'z2', 'z3', 'z4', 'z5', 'z6', 'z7', 'z8', 'z9',
                        'c1', 'c2', 'c3', 'c4', 'c5', 'c6', 'c7', 'c8', 'c9',
                        'a1', 'a2', 'a3', 'a4', 'a5', 'a6', 'a7', 'a8', 'a9',
                        's1', 's2', 's3', 's4', 's5', 's6', 's7', 's8', 's9', 'f1'])
ssc

In [ ]:
# 위치 데이터와 학생 수 데이터를 합치고 csv로 저장
import pandas as pd

# 초등학교
eslc = pd.merge(esl, esc, on='학교명')
eslc = eslc.drop(columns=['정보공시 학교코드_y', '교육지원청_y'])
eslc.to_csv('c:\\data\\es_lc.csv', index=False)

# 중학교
mslc = pd.merge(msl, msc, on='학교명')
mslc = mslc.drop(columns=['정보공시 학교코드_y', '교육지원청_y'])
mslc.to_csv('c:\\data\\ms_lc.csv', index=False)

# 고등학교
hslc = pd.merge(hsl, hsc, on='학교명')
hslc = hslc.drop(columns=['정보공시 학교코드_y', '교육지원청_y'])
hslc.to_csv('c:\\data\\hs_lc.csv', index=False)

# 특수학교
sslc = pd.merge(ssl, ssc, on='학교명')
sslc = sslc.drop(columns=['정보공시 학교코드_y', '교육지원청_y'])
sslc.to_csv('c:\\data\\ss_lc.csv', index=False)

# 초, 중, 고, 특수 학교 테이블을 합친 코드
sj_sch = pd.concat([eslc, mslc, hslc, sslc], ignore_index=True)
sj_sch.to_csv('c:\\data\\sj_sch.csv')

# <통근 인구>

### 해당동의 종사자 수를 그리드로 나눠서 단일 점에 집중되지 않도록 함

### 격자 포인트 생성 방법

### 1. 구글 지도에서 레이어에 좌표를 찍어서 파일로 내보내기 한다. (kml 파일로 다운로드)
### 2. GPS Visualizer 사이트에서 kml 로 받은 파일을 txt 파일로 변환한다
### 3. txt 파일을 이용해 격자 포인트를 생성

In [ ]:
import folium
import pandas as pd
import numpy as np
from shapely.geometry import Polygon, Point
import os

# 파일이 위치한 경로 설정
file_path = 'c:\\data\\동별_종사자\\'

# 경로 내의 모든 .txt 파일 가져오기
file_list = [f for f in os.listdir(file_path) if f.endswith('.txt')]

# folium 지도 생성 (중심 위치와 초기 줌 설정)
center_lat = 36.5185  # 중심 위도 (예시)
center_lon = 127.2325  # 중심 경도 (예시)
m = folium.Map(location=[center_lat, center_lon], zoom_start=14)

# 각 파일을 불러와 지도에 추가하는 작업
for file_name in file_list:
    df = pd.read_csv(os.path.join(file_path, file_name), sep='\t')

    # 파일 내의 위도 및 경도 값으로 다각형 설정
    min_lat = df['latitude'].min()
    max_lat = df['latitude'].max()
    min_lon = df['longitude'].min()
    max_lon = df['longitude'].max()

    # 좌표 4개를 사용해 다각형 영역 설정 (최대, 최소값을 기준으로)
    polygon_points = [
        (max_lat, min_lon),  # 좌상단
        (max_lat, max_lon),  # 우상단
        (min_lat, max_lon),  # 우하단
        (min_lat, min_lon)   # 좌하단
    ]
    
    # 다각형 생성
    polygon = Polygon(polygon_points)

    # 격자 크기 설정 (0.001도 간격 -> 약 100m)
    grid_size = 0.001

    # 위도와 경도의 범위 설정
    lat_range = np.arange(min_lat, max_lat, grid_size)
    lon_range = np.arange(min_lon, max_lon, grid_size)

    # 격자 포인트 생성
    grid_points = [(lat, lon) for lat in lat_range for lon in lon_range]

    # 다각형 내부에 있는 격자 포인트 필터링
    filtered_grid_points = [point for point in grid_points if polygon.contains(Point(point))]

    # 필터링된 격자 포인트 지도에 추가
    for point in filtered_grid_points:
        folium.CircleMarker(location=[point[0], point[1]], radius=3, color='blue', fill=True).add_to(m)

    # 원래 위치 데이터 지도에 추가
    for idx, row in df.iterrows():
        folium.Marker(location=[row['latitude'], row['longitude']], popup=row['name'], icon=folium.Icon(color='green')).add_to(m)

    # 다각형 영역 지도에 추가
    folium.PolyLine(locations=polygon_points + [polygon_points[0]], color='red').add_to(m)

# 지도를 HTML 파일로 저장 또는 브라우저에서 직접 열기
m.save('combined_grid_map.html')
m
